In [1]:
import os
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, Dense

Using TensorFlow backend.


In [2]:
maxlen = 30 #sequence length
sentences = [] #store extracted sentences
next_chars = [] #for every extracted sentences, store its next char as target

In [3]:
with open('../input/exampleText_shorter', 'r', encoding = 'utf8') as fr:
    lines = fr.read() #read all chars
    for i in range(0, len(lines) - maxlen):
        sentences.append(lines[i:i+maxlen])
        next_chars.append(lines[i+maxlen])
print('number of extracted sentences %d' % len(sentences))

number of extracted sentences 80116


In [4]:
chars = sorted(list(set(lines))) #all unique chars
chars_indices = dict((char, i) for i, char in enumerate(chars))
print('number of unique chars %d' % len(chars))

number of unique chars 2671


In [5]:
num_samples = len(sentences)
num_chars = len(chars)

In [6]:
model = Sequential()
model.add(LSTM(256, input_shape=(maxlen, num_chars)))
model.add(Dense(num_chars, activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer = 'rmsprop')

Instructions for updating:
Colocations handled automatically by placer.


In [7]:
# input, shape = [size, maxlen, num_chars]
x = np.zeros((num_samples, maxlen, num_chars), dtype=np.bool)
# target, shape = [size, num_chars]
y = np.zeros((num_samples, num_chars), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for j, ch in enumerate(sentence):
        x[i, j, chars_indices[ch]] = 1
    y[i, chars_indices[next_chars[i]]] = 1

In [8]:
model.fit(x, y, epochs=100, batch_size=1024)

Instructions for updating:
Use tf.cast instead.
Epoch 1/100
80116/80116 [==============================] - 67s 842us/step - loss: 6.1030
Epoch 2/100
80116/80116 [==============================] - 63s 785us/step - loss: 5.9384
Epoch 3/100
80116/80116 [==============================] - 63s 787us/step - loss: 5.8334
Epoch 4/100
80116/80116 [==============================] - 63s 785us/step - loss: 5.7153
Epoch 5/100
80116/80116 [==============================] - 63s 787us/step - loss: 5.5792
Epoch 6/100
80116/80116 [==============================] - 63s 786us/step - loss: 5.4319
Epoch 7/100
80116/80116 [==============================] - 63s 786us/step - loss: 5.2916
Epoch 8/100
80116/80116 [==============================] - 63s 785us/step - loss: 5.1592
Epoch 9/100
80116/80116 [==============================] - 63s 783us/step - loss: 5.0353
Epoch 10/100
80116/80116 [==============================] - 63s 784us/step - loss: 4.9213
Epoch 11/100
80116/80116 [==============================] - 6

In [9]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    
    probabs = np.random.multinomial(1, preds, 1)
    return np.argmax(probabs)

In [10]:
def write(model, temperature, word_num, begin_sentence):
    inp = begin_sentence[:maxlen] #initial input defined by user
    result = inp + '\\\ '
    for _ in range(word_num):
        sampled = np.zeros((1, maxlen, num_chars))
        for i, ch in enumerate(inp):
            sampled[0, i, chars_indices[ch]] = 1.0
        
        preds = model.predict(sampled, verbose=0)[0]
        if temperature is None:
            next_word = chars[np.argmax(preds)]
        else:
            next_index = sample(preds, temperature)
            next_word = chars[next_index]
        
        inp += next_word
        inp = inp[1:] #remove first word
        result += next_word
    return result
        

In [11]:
#test
begin_sentence = lines[50003:50100]
print('start sentence: ', begin_sentence[:maxlen])

#no temperature
print('no temperature')
print(write(model, None, 200, begin_sentence))

#various temperature
for temp in [0.5, 1.0, 1.5]:
    print('temperature %f' % temp)
    print(write(model, temp, 200, begin_sentence))

start sentence:  
一块的红布腰带来。“有这个，我就饿不着！”说完，他赶紧把小
no temperature

一块的红布腰带来。“有这个，我就饿不着！”说完，他赶紧把小\\ 褂又扣好。

    ““听吧！”二哥看出亲接的飞飞”。把把然来有时多没法，他可叫叫姑母上发了几句，才 坐不
从，并且给他便口口中，几个借借的话话，可是进进来。虽然在生的活里，象 位
人了些小孩子的金金个金金的，
来，能决什么情么有的那样子，在我的一下不会。他那样，由又对他回 去。
家。作己他路在他的时候，他才起来了点些年。并不过他是对年领 的理物。是
他明知是短人管牧是那么，一个字而小个小旧儿子
temperature 0.500000

一块的红布腰带来。“有这个，我就饿不着！”说完，他赶紧把小\\ 褂又扣好。

    ““啊？”二毛子打二了出鼻子。他看到街上门里了多小儿子，他上了下了。多甫 去

  十下的吃明饭，我不能说：他说决定：什么，“行吃吃！不多，还是少官吗？”

    定大爷，没有点不过！就是这么，都还是一心！我找吧！”

    “那了？老白呢？你看出这种就不大半气！”

    “轿国！多老老而！”父亲点，吹来更点了。一看。王掌柜，他不是的头 很大爷，可是，他觉得点很自己行
temperature 1.000000

一块的红布腰带来。“有这个，我就饿不着！”说完，他赶紧把小\\ 褂 扣
好，而“话看：“叫这儿行！别人！一须下来计造了呢？”“我们这不会说，定大点事来！他就是他 什么：
说，自己三不十分一个（你也有个可能不该艺品地与他跟人！你家一闹？她 等二爷等拿了世 ，只拿到十四明放而 钱
严。
  很 十分脸空地回了他的光往他想起。如叫“先换洋人人说的声，他该自己吃不好，第 花
儿自己得九件肉，我了手扯大—也恭敬实①是大，要才一定大爷，身旁 刷
前的没大倒正的金于桌天就诉
temperature 1.500000

一块的红布腰带来。“有这个，我就饿不着！”说完，他赶紧把小\\ 光又扣好。

    ““别？是您”，对！过十成冲出眼看，把按带死“分还记给我的那个么炸！”他 “血
显管子是他还缘冷等不肯吗？不多陪二便不管歉！怎么丢他舅舅的颇多认都有水红并的是不席下，可以上以如兴 进
既是枣关，衙汤大烟喝过出体来。如
商啊也要不酱不把规对，大乱事是样：路使